In [1]:
# Imports All relevant libraries
from super_func import *


In [2]:
# opens the file to the api keys saved as creds
with open('.hidden/creds.json') as f:
    creds = json.load(f)

In [3]:
# Sets variables for url_params
term = 'Pet Store'
location = 'Queens'
limit = 50
offset = 0
# Sets variables for csv file names for business
csv_filename = './csv/petstores.csv'
csv_filepath = './csv/petstores.csv'
# Sets variables for csv file names for reviews
reviews_filename = './csv/reviews.csv'

In [4]:
# Url argument
url_params = {'term': term,
        'location': location,
        'limit': limit,
        'offset': offset}

In [5]:
# Deletes the current csv file 
if os.path.isfile(csv_filename):
    os.remove(csv_filename)
# Deletes the current csv file 
if os.path.isfile(csv_filename):
    os.remove(csv_filename)

In [6]:
# create a variable  to keep track of which result you are in. 
cur = 0
num = 150
#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, creds)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    df_save(parsed_results, str(csv_filename))
    #increment the counter by 50 to move on to the next results
    cur += 50

In [7]:
# Lets you store the csv in a pandas DF
pet_stores = pd.read_csv('./csv/petstores.csv', index_col=0)

In [8]:
# Deletes the current csv file 
if os.path.isfile(reviews_filename):
    os.remove(reviews_filename)

In [9]:
business_ids, business_ids_names, business_names = business_ids_get()

In [14]:
reviews = yelp_call_reviews(creds, business_ids)

In [15]:
list_of_reviews = list_of_reviews(reviews,business_ids, business_ids_names, business_names)

In [16]:
reviews_df = pd.DataFrame(list_of_reviews)
reviews_selection = reviews_df[['business_id', 'text_excerpt', 'review_url',  'time_created', 'user_name']]
reviews_cleaned = reviews_selection.rename(columns={"business_id": "id"})

In [17]:
df_save(reviews_cleaned, reviews_filename)

In [18]:
pet_stores_csv = pd.read_csv('./csv/petstores.csv', index_col= 0)
reviews_csv = pd.read_csv('./csv/reviews.csv', index_col= 0)

In [19]:
merged_result = pd.merge(pet_stores_csv, reviews_csv, on="id")

**The Map**

In [ ]:
base_map = folium.Map([lat,long] , zoom_start=11)

In [60]:

Qresults = merged_result[merged_result['zip'].isin(Qzips)]
Mresults = merged_result[merged_result['zip'].isin(Mzips)]

vet_zips = pd.read_csv('./csv/vets.csv')

VQresults = vet_zips[vet_zips['zip_code'].isin(Qzips)]
VMresults = vet_zips[vet_zips['zip_code'].isin(Mzips)]

In [61]:
# Queens Points
QPoints = folium.FeatureGroup(name='Queens', show=True)
base_map.add_child(QPoints)

for (index, row) in Qresults.iterrows():
    folium.Marker(location=[row.loc['latitude'], row.loc['longitude']], 
                  popup=row.loc['name'], 
                  icon=folium.Icon(color='blue', icon='fa-cart-plus', prefix='fa'),
                  tooltip='click').add_to(QPoints)

for (index, row) in VQresults.iterrows():
    folium.Marker(location=[row.loc['latitude'], row.loc['longitude']], 
                  popup=row.loc['business_name'], 
                  icon=folium.Icon(color='pink', icon='fa-heart', prefix='fa'),
                  tooltip='click').add_to(QPoints)
# Manhattan Points 
MPoints = folium.FeatureGroup(name='Manhattan', show=True)
base_map.add_child(MPoints)

for (index, row) in Mresults.iterrows():
    folium.Marker(location=[row.loc['latitude'], row.loc['longitude']], 
                  popup=row.loc['name'], 
                  icon=folium.Icon(color='blue', icon='fa-cart-plus', prefix='fa'),
                  tooltip='click').add_to(MPoints)

for (index, row) in VMresults.iterrows():
    folium.Marker(location=[row.loc['latitude'], row.loc['longitude']], 
                  popup=row.loc['business_name'], 
                  icon=folium.Icon(color='pink', icon='fa-heart', prefix='fa'),
                  tooltip='click').add_to(MPoints)

# Extra Functionality
folium.GeoJson(geo_data_Q, name='Queens').add_to(QPoints)
folium.GeoJson(geo_data_M, name='Manhattan').add_to(MPoints)
folium.raster_layers.TileLayer('Open Street Map').add_to(base_map)
folium.raster_layers.TileLayer('CartoDB Dark_Matter').add_to(base_map)
folium.LayerControl().add_to(base_map)
plugins.ScrollZoomToggler().add_to(base_map)

In [62]:
base_map
    